In [24]:
#imports
import math
import random
import pandas as pd
import numpy as np
import os
import networkx as nx

In [25]:
excel_list1 = os.listdir("Raw_Data")
excel_list = []
for i in excel_list1:
    if(i.endswith(".csv")):
        excel_list.append(i)      

In [30]:
graphs = []
for i in excel_list:
    path = "Raw_Data/"+i
    if(i=="0815.csv"):
        excel = pd.read_csv(path ,header=0, usecols = [0,1,2,3] ,names=["city1","city2","to","from"] )
    else:
        excel = pd.read_csv(path ,header=0, index_col=0, usecols = [0,1,2,3,4] ,names=["SNo","city1","city2","to","from"] )
    G = nx.Graph()
    print(path)
    for i in range(1,len(excel)):
        weight1 = excel["to"][i]
        weight2 = excel["from"][i]
        G.add_edge(excel["city1"][i],excel["city2"][i],weight=weight1)
        G.add_edge(excel["city2"][i],excel["city1"][i],weight=weight2)
    print(nx.adjacency_matrix(G).todense())
    graphs.append(G)

Raw_Data/0116.csv
[[  0 854   0 ...   0   0   0]
 [854   0   0 ...   0   0 979]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0 979   0 ...   0   0   0]]
Raw_Data/0117.csv
[[   0 1234    0 ...    0    0    0]
 [1234    0    0 ...    0    0  468]
 [   0    0    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0  468    0 ...    0    0    0]]
Raw_Data/0118.csv
[[  0  56   0 ...   0   0   0]
 [ 56   0 892 ...   0   0   0]
 [  0 892   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
Raw_Data/0119.csv
[[    0  2307     0 ...     0     0     0]
 [ 2307     0  1634 ...     0     0 20222]
 [    0  1634     0 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0 20222     0 ...     0     0     0]]
Raw_Data/0120.csv
[[   0 2248    0 ...    0    0    

[[   0 2205    0 ...    0    0    0]
 [2205    0 2512 ...    0    0    0]
 [   0 2512    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]
Raw_Data/0719.csv
[[   0 2088    0 ...    0    0    0]
 [2088    0 1214 ...    0    0    0]
 [   0 1214    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]
Raw_Data/0720.csv
[[    0  1512  3876 ...     0     0     0]
 [ 1512     0 19506 ...     0     0     0]
 [ 3876 19506     0 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]
Raw_Data/0815.csv
[[    0    12    76 ...     0     0     0]
 [   12     0 27860 ...     0     0 13899]
 [   76 27860     0 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    